# How long has each report been running?

**We are still considering active reports!**

---

>Questions being answered in this notebook.
>- [x] What is the execution period of these reports?
>- [x] Which reports have been running the longest?
>- [x] Which reports have been running the longest by period (since `LastModifiedDate`)?


In [1]:
import pandas as pd
import datetime as dt

Loading active reports data.

In [2]:
active_reports = pd.read_csv("datasets/active_reports.csv")

C:\Users\jefer\AppData\Local\Temp\ipykernel_7640\2173927402.py:1: DtypeWarning: Columns (10,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  active_reports = pd.read_csv("datasets/active_reports.csv")


## 1. What is the execution period of these reports?

Building a function to calculate the period of days between the `LastRunDate` and the last change date (`LastModifiedDate`) for each report.

In [3]:
ref_date = dt.date(2022, 6, 20)

In [4]:
def running_days_since_creation(report):
    timedelta = ref_date - dt.datetime.fromisoformat(report.CreatedDate).date()
    return timedelta.days + 1 # fix to get reports with ReferenceDate == CreatedDate

In [5]:
def running_days_since_last_modification(report):
    timedelta = ref_date - dt.datetime.fromisoformat(report.LastModifiedDate).date()
    return timedelta.days + 1 # fix to get reports with ReferenceDate == LastModifiedDate

Adding a new column to store `DaysSinceCreation` and `DaysSinceLastModifiedDate` information.

In [6]:
active_reports['DaysSinceCreation'] = active_reports\
    .apply(lambda r: running_days_since_creation(r), axis=1)

active_reports['DaysSinceLastModifiedDate'] = active_reports\
    .apply(lambda r: running_days_since_last_modification(r), axis=1)

The following is a preview of the new dataframe we will be getting from this new info.

In [7]:
active_reports[['REPORT_ID_DERIVED', 'Name', 'LastRunDate', 'DaysSinceCreation']]\
    .groupby('REPORT_ID_DERIVED')\
    .agg({'Name': 'first', 'LastRunDate': 'count', 'DaysSinceCreation': 'first'})\
    .rename(columns={'LastRunDate': 'RunCount', 'Name': 'ReportName'})\
    .sort_values('RunCount', ascending=False)\
    .head(10)

,ReportName,RunCount,DaysSinceCreation,DaysSinceLastModifiedDate
REPORT_ID_DERIVED,,,,
00O0b000006iNwdEAE,New Email By Team - w\o Sup filter,8816,1130,103
00O0b000004kTazEAE,My Cases and Tasks,6708,1452,22
00O2R000004AmwmUAC,S&AS Reactive Queues,2347,578,106
00O2R000003JCXaUAO,New Email By Team_HYD,2293,735,304
00O6P000000ZMcoUAG,Wipro - VxRail Case Title Report,1392,132,132
00O0b000005yLBGEA2,"Cases with Tasks, Calendar & Reminders",1026,1110,307
00O2R000004AmgKUAS,My Team's Cases Without Tasks V1.0,926,612,82
00O0b000004AmRrEAK,Chat Average Handle Time,893,1591,766
00O2R0000043l7WUAQ,Case Aging Report,768,858,319


From this we can evaluate how often each report was run during each period.

In [8]:
runtime_span = active_reports[['REPORT_ID_DERIVED', 'Name', 'LastRunDate', 'DaysSinceCreation', 'DaysSinceLastModifiedDate']]\
    .groupby('REPORT_ID_DERIVED')\
    .agg({'Name': 'first', 'LastRunDate': 'count', 'DaysSinceCreation': 'first', 'DaysSinceLastModifiedDate': 'last'})\
    .rename(columns={'LastRunDate': 'RunCount', 'Name': 'ReportName'})\
    .sort_values('RunCount', ascending=False)\
    .reset_index()

In [9]:
runtime_span.DaysSinceCreation.mean()

499.9533527696793

In [10]:
runtime_span.DaysSinceLastModifiedDate.mean()

294.6530612244898

In [11]:
# most recent report and the oldest one
runtime_span.DaysSinceCreation.min(), runtime_span.DaysSinceCreation.max()

(18, 1591)

In order to sort the reports by the highest running period of execution. We can either consider the most executed reports in the last 30 days or, in addition, consider the period in which the executions took place. Let's investigate both.

## 2. Reports that have been running the longest

In [12]:
runtime_span.sort_values('DaysSinceCreation', ascending=False).head()

,REPORT_ID_DERIVED,ReportName,RunCount,DaysSinceCreation,DaysSinceLastModifiedDate
66,00O0b000004AmRmEAK,Completed Chat Sessions,54,1591,857
274,00O0b000004AmSCEA0,Cases by Age & Status,2,1591,319
114,00O0b000004AmS2EAK,Overdue Tasks by Case,18,1591,857
62,00O0b000004AmRyEAK,Cases by Status,63,1591,585
52,00O0b000004AmS0EAK,Cases by Priority,79,1591,157


In [13]:
running_the_longest = runtime_span.sort_values('DaysSinceCreation', ascending=False)
running_the_longest.to_csv("./datasets/active_reports_that_have_been_running_the_longest.csv", index=False)

In [14]:
running_the_longest['AvgRunsOverLast30days'] = running_the_longest.RunCount / 30

In [15]:
running_the_longest\
    .drop(columns=['REPORT_ID_DERIVED', 'DaysSinceLastModifiedDate'])\
    .sort_values('RunCount', ascending=False)\
    .head()

,ReportName,RunCount,DaysSinceCreation,AvgRunsOverLast30days
0,New Email By Team - w\o Sup filter,8816,1130,293.866667
1,My Cases and Tasks,6708,1452,223.600000
2,S&AS Reactive Queues,2347,578,78.233333
3,New Email By Team_HYD,2293,735,76.433333
4,Wipro - VxRail Case Title Report,1392,132,46.400000


## 3. Reports that are running longer per period

<p style='color:red'>FIX: need loads active_reports_since_last_modified_date here.


We can create a ratio between the number of runs and the period in order to estimate that. Let's define a `AvgDailyRun`.

In [16]:
runtime_span['AvgRunsOverLast30daysSinceLastModifiedDate'] =\
    runtime_span.RunCount / (30 - runtime_span.DaysSinceLastModifiedDate)

In [17]:
runtime_span.sort_values('DaysSinceLastModifiedDate', ascending=False).head()

,REPORT_ID_DERIVED,ReportName,RunCount,DaysSinceCreation,DaysSinceLastModifiedDate,AvgRunsOverLast30daysSinceLastModifiedDate
126,00O0b000004fSjgEAE,Open Tasks (Blue Steel) Osp,14,1522,1522,-0.009383
172,00O0b000004fSjXEAU,DELL Cases by Status,6,1522,1522,-0.004021
92,00O0b000004fSjdEAE,Cases by Status Osp,27,1522,1522,-0.018097
80,00O0b000004kKJKEA2,Wrap Up Time,38,1513,1513,-0.025624
91,00O0b000004kf2wEAA,Email Queue WIP,27,1362,1250,-0.022131


In [19]:
# running_the_longest_in_period = runtime_span.sort_values('DaysSinceLastModifiedDate', ascending=False)
# running_the_longest_in_period.to_csv("./datasets/active_reports_that_have_been_running_the_longest_in_period.csv", index=False)